In [1]:
import os
import getpass
from dotenv import load_dotenv
import langgraph

load_dotenv()

langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")
tavily_api_key = os.getenv("TAVILY_API_KEY")

Model

In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
tools = [search]

In [29]:
# 확인
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

# Bedrock 모델 설정
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0)
)

# 모델 호출 확인
# example_result = model.invoke([HumanMessage(content="다음 미국 대선 후보는 누구인가요?")])
# example_result

Parser

In [21]:
from langchain_core.output_parsers import StrOutputParser

# 출력 파서 설정
parser = StrOutputParser()

# 파서 확인
# parsed_result = parser.invoke(example_result)
# print(parsed_result)  # 출력: '안녕!'

Prompt Template

In [30]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 템플릿
system_template = (
    "당신은 질문에 답하는 어시스턴트입니다. "
    "다음 문서에서 제공된 내용을 사용하여 질문에 답변하세요. "
    "모르면 모른다고 말하세요. 최대 세 문장으로 간결하게 답하세요."
    "\\n\\n"
    "{context}"
)

# 프롬프트 템플릿
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        ("user", "{input}"),
    ]
)

# # 템플릿 호출
# prompt_result = prompt_template.invoke()
# print(prompt_result.to_messages())

pdf

In [31]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 경로 설정
file_path = "./nike-10k-2023.pdf"

# PyPDFLoader를 사용하여 PDF 파일 로드
loader = PyPDFLoader(file_path)

# PDF 문서를 로드하고 각 페이지의 내용을 저장
docs = loader.load()

In [ ]:
from langchain_aws import BedrockEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 스플리터 설정 (1000자 단위로 문서를 나누고, 중첩 200자 적용)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 벡터 스토어에 임베딩 적용 및 문서 저장
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=BedrockEmbeddings(
        model_id='amazon.titan-embed-text-v1',
    )
)

# 리트리버 생성
# retriever = vectorstore.as_retriever()

In [33]:
from langgraph.graph import START, MessagesState, StateGraph
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

def create_qna_chain(model, prompt_template, parser):
    return prompt_template | model | parser
qna_chain = create_qna_chain(model, prompt_template, parser)
rag_chain = create_retrieval_chain(retriever, qna_chain)


In [39]:
results = rag_chain.invoke({"input": "내가 무슨 질문을 했었지?"})
results['answer']

'죄송합니다만, 이전에 어떤 질문을 하셨는지에 대한 정보가 주어진 문서에 없습니다. 제가 이전 대화 내용을 알지 못하기 때문에 귀하의 이전 질문을 확인할 수 없습니다.'

In [43]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="삼성전자 주가가 어떻게 되나요?")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="내가 전에 뭐 물어봤지?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 417, 'completion_tokens': 109, 'total_tokens': 526}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 417, 'completion_tokens': 109, 'total_tokens': 526}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-1790042a-99cd-47ea-b5aa-b7e987a41187-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '삼성전자 현재 주가'}, 'id': 'toolu_bdrk_01WDZkHvuMo32dHLNLtZ8izL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 417, 'output_tokens': 109, 'total_tokens': 526})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.yna.co.kr/view/AKR20211231126500002", "content": "삼성전자 주가는 2021년 연간 수익률 마이너스로 부진하며 7만원대로 마무리했다. 그러나 2022년에는 메모리 반도체 업사이클에 힘입어 목표주가를 10만원에서 12만원으로 올리는 증권가들의 의견이 많다."}, {"url": "https://alphasquare.co.kr/home/stock-summary?code=005